In [97]:
# # 재시작 없이 import 파일 자동 적용
# %load_ext autoreload
# %autoreload 2

---
# Private Test Set - Inference

In [1]:
import os
import h5py
import random
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tqdm import tqdm
from sklearn.metrics import f1_score
from sklearn.metrics import roc_auc_score
from sklearn.metrics import matthews_corrcoef
import torch
import torch.nn as nn
from torch.utils.data import DataLoader

from config import CONFIG
from data_factory import collate_fn
from data_loader import PublicTest
from classification import Exp_Classification

In [2]:
torch.cuda.empty_cache()

### Checkpoints 및 CONFIG file 경로 설정

In [3]:
# import importlib.util

# test_folder_name = 'classification_T#17_Medformer_K-Medicon_bs8_sl96_lr0.0001_pl96_dm128_nh8_el6_df256_fc1_ebtimeF_Exp_seed41'
# test_id = test_folder_name.split('_')[1]
# print("Test ID:", test_id)
# # test_config_path = f"./checkpoints/{test_folder_name}/config_{test_id}.py"
# test_config_path = f"./checkpoints/{test_folder_name}/config_{test_id}.py"

# # 모듈 이름 생성
# module_name = os.path.splitext(os.path.basename(test_config_path))[0]

# # 모듈 로드
# spec = importlib.util.spec_from_file_location(module_name, test_config_path)
# config_module = importlib.util.module_from_spec(spec)
# spec.loader.exec_module(config_module)

# # CONFIG 변수 사용
# CONFIG = config_module.CONFIG  # config_T#01.py 파일 내의 CONFIG 변수

In [4]:
# print(CONFIG)

In [5]:
print(CONFIG['model_id'])
seed = CONFIG['seed']
    
random.seed(seed)
os.environ['PYTHONHASHSEED'] = str(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True

S01_11


In [6]:
setting = "{}_{}_{}_{}_bs{}_{}hz_{}_{}_sl{}_lr{}_pl{}_dm{}_nh{}_el{}_df{}_fc{}_eb{}_{}_{}_seed{}_{}".format(
                CONFIG['task_name'],
                CONFIG['model_id'],
                CONFIG['model'],
                CONFIG['data'],
                CONFIG['batch_size'],
                CONFIG['signal_hz'],
                CONFIG['monitor'],
                CONFIG['loss'],
                CONFIG['seq_len'],
                CONFIG['learning_rate'],
                CONFIG['pred_len'],
                CONFIG['d_model'],
                CONFIG['n_heads'],
                CONFIG['e_layers'],
                CONFIG['d_ff'],
                CONFIG['factor'],
                CONFIG['embed'],
                CONFIG['activation'],
                CONFIG['des'],
                CONFIG['seed'],
                CONFIG['augmentations'],
            )

exp = Exp_Classification(CONFIG)

Use GPU: cuda:0


In [7]:
# pth_list = os.listdir('./private_test_pth')
# print(len(pth_list))
# pth_list

In [8]:
# model_path = f"./checkpoints/{test_folder_name}/checkpoint.pth"
model_path = f"./private_test_pth/{CONFIG['model_id']}.pth"
print(model_path)

exp.swa_model.load_state_dict(torch.load(model_path))
exp.swa_model.eval()

./private_test_pth/S01_11.pth


AveragedModel(
  (module): Model(
    (enc_embedding): ListPatchEmbedding(
      (value_embeddings): ModuleList(
        (0): CrossChannelTokenEmbedding(
          (tokenConv): Conv2d(1, 128, kernel_size=(12, 2), stride=(1, 2), bias=False, padding_mode=circular)
        )
        (1): CrossChannelTokenEmbedding(
          (tokenConv): Conv2d(1, 128, kernel_size=(12, 4), stride=(1, 4), bias=False, padding_mode=circular)
        )
        (2-3): 2 x CrossChannelTokenEmbedding(
          (tokenConv): Conv2d(1, 128, kernel_size=(12, 8), stride=(1, 8), bias=False, padding_mode=circular)
        )
        (4-7): 4 x CrossChannelTokenEmbedding(
          (tokenConv): Conv2d(1, 128, kernel_size=(12, 16), stride=(1, 16), bias=False, padding_mode=circular)
        )
        (8-15): 8 x CrossChannelTokenEmbedding(
          (tokenConv): Conv2d(1, 128, kernel_size=(12, 32), stride=(1, 32), bias=False, padding_mode=circular)
        )
      )
      (position_embedding): PositionalEmbedding()
      

---
### 250 hz

In [9]:
# Public label
testset = PublicTest("./dataset/Signal_Test_Private_250hz.h5")
public_test_loader = DataLoader(
    testset,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    drop_last=False,
    # collate_fn=lambda x: collate_fn(
    #             x, max_len=CONFIG['seq_len']),
    )

In [10]:
device = torch.device("cuda:0")

total_loss = []
preds = []
trues = []

with torch.no_grad():
    for i, (batch_x, label) in enumerate(tqdm(public_test_loader)):
        batch_x = batch_x.float().to(device)
        label = label.to(device)
        # padding_mask = padding_mask.float().to(device)

        outputs = exp.swa_model(batch_x, None, None, None)

        pred = outputs.detach().cpu()

        preds.append(outputs.detach())
        trues.append(label)

preds = torch.cat(preds, 0)
trues = torch.cat(trues, 0)

probs = torch.nn.functional.softmax(
# probs = torch.nn.functional.sigmoid(
    preds
)  # (total_samples, num_classes) est. prob. for each class and sample

trues_onehot = (torch.nn.functional.one_hot(trues.reshape(-1,).to(torch.long),
                num_classes=CONFIG['num_class'],).float().cpu().numpy())

print(trues_onehot.shape)
predictions = (
    torch.argmax(probs, dim=1).cpu().numpy()
)  # (total_samples,) int class index for each sample
probs = probs.cpu().numpy()
trues = trues.flatten().cpu().numpy()

100%|██████████| 300/300 [00:10<00:00, 27.95it/s]

(300, 2)


In [11]:
# softmax
f1 = f1_score(trues, predictions, average="binary")
auroc = roc_auc_score(trues_onehot, probs)
# auroc = roc_auc_score(trues, probs)
mcc = matthews_corrcoef(trues, predictions)
cpi = (0.25 * f1) + (0.25 * auroc) + (0.5 * mcc)

print(f"F1 score: {f1}")
print(f"AUROC score: {auroc}")
print(f"MCC score: {mcc}")
print(f"CPI score: {cpi}")
print()
print(f"0개수: {len(np.where(predictions[:]==0)[0])}")
print(f"1개수: {len(np.where(predictions[:]==1)[0])}")

F1 score: 0.6013986013986015
AUROC score: 0.8278518518518518
MCC score: 0.47805103537939936
CPI score: 0.596338131002313

0개수: 232
1개수: 68


In [12]:
# df = pd.DataFrame({"0":probs[:,0], "1":probs[:,1]})
df = pd.DataFrame({"Probability":probs[:,1]})

# save_xlsx = pd.ExcelWriter(f"./RTFACT_Public_Possibility_{CONFIG['model_id']}.xlsx")
# df.to_excel(save_xlsx, index = False) # xlsx 파일로 변환
# save_xlsx.save() #xlsx 파일로 저장

# ExcelWriter 사용
with pd.ExcelWriter(f"../../Submission results/private/RTFACT_241014_Private_Possibility_{CONFIG['model_id']}_Softmax.xlsx", engine='openpyxl') as writer:
# with pd.ExcelWriter(f"../../Submission results/private/RTFACT_241014_Private_Possibility_{CONFIG['model_id']}_Sigmoid.xlsx", engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='Sheet1', index=False)

---
### 500 hz

In [6]:
# # Public label
# testset = PublicTest("./dataset/Signal_Test_Private_500hz.h5")
# public_test_loader = DataLoader(
#     testset,
#     batch_size=1,
#     shuffle=False,
#     num_workers=0,
#     drop_last=False,
#     # collate_fn=lambda x: collate_fn(
#     #             x, max_len=CONFIG['seq_len']),
#     )

In [7]:
# device = torch.device("cuda:0")

# total_loss = []
# preds = []
# trues = []

# with torch.no_grad():
#     for i, (batch_x, label) in enumerate(tqdm(public_test_loader)):
#         batch_x = batch_x.float().to(device)
#         label = label.to(device)
#         # padding_mask = padding_mask.float().to(device)

#         outputs = exp.swa_model(batch_x, None, None, None)

#         pred = outputs.detach().cpu()

#         preds.append(outputs.detach())
#         trues.append(label)

# preds = torch.cat(preds, 0)
# trues = torch.cat(trues, 0)

# probs = torch.nn.functional.softmax(
#     preds
# )  # (total_samples, num_classes) est. prob. for each class and sample

# trues_onehot = (torch.nn.functional.one_hot(trues.reshape(-1,).to(torch.long),
#                 num_classes=CONFIG['num_class'],).float().cpu().numpy())

# print(trues_onehot.shape)
# predictions = (
#     torch.argmax(probs, dim=1).cpu().numpy()
# )  # (total_samples,) int class index for each sample
# probs = probs.cpu().numpy()
# trues = trues.flatten().cpu().numpy()

100%|██████████| 300/300 [00:15<00:00, 19.42it/s]

(300, 2)


In [10]:
# # softmax
# f1 = f1_score(trues, predictions, average="binary")
# auroc = roc_auc_score(trues_onehot, probs)
# # auroc = roc_auc_score(trues, probs)
# mcc = matthews_corrcoef(trues, predictions)
# cpi = (0.25 * f1) + (0.25 * auroc) + (0.5 * mcc)

# print(f"F1 score: {f1}")
# print(f"AUROC score: {auroc}")
# print(f"MCC score: {mcc}")
# print(f"CPI score: {cpi}")
# print()
# print(f"0개수: {len(np.where(predictions[:]==0)[0])}")
# print(f"1개수: {len(np.where(predictions[:]==1)[0])}")

F1 score: 0.4968944099378882
AUROC score: 0.7535999999999999
MCC score: 0.3149306866445732
CPI score: 0.47008894580675864

0개수: 214
1개수: 86


In [11]:
# # df = pd.DataFrame({"0":probs[:,0], "1":probs[:,1]})
# df = pd.DataFrame({"Probability":probs[:,1]})

# # save_xlsx = pd.ExcelWriter(f"./RTFACT_Public_Possibility_{CONFIG['model_id']}.xlsx")
# # df.to_excel(save_xlsx, index = False) # xlsx 파일로 변환
# # save_xlsx.save() #xlsx 파일로 저장

# # ExcelWriter 사용
# # with pd.ExcelWriter(f"../../Submission results/RTFACT_241011_Public_Possibility_{CONFIG['model_id']}_Sigmoid.xlsx", engine='openpyxl') as writer:
# with pd.ExcelWriter(f"../../Submission results/private/RTFACT_241014_Private_Possibility_{CONFIG['model_id']}_Softmax.xlsx", engine='openpyxl') as writer:
#     df.to_excel(writer, sheet_name='Sheet1', index=False)

---
### 62.5 hz

In [ ]:
# Public label
testset = PublicTest("./dataset/Signal_Test_Private_quarter.h5")
public_test_loader = DataLoader(
    testset,
    batch_size=1,
    shuffle=False,
    num_workers=0,
    drop_last=False,
    # collate_fn=lambda x: collate_fn(
    #             x, max_len=CONFIG['seq_len']),
    )

In [ ]:
device = torch.device("cuda:0")

total_loss = []
preds = []
trues = []

with torch.no_grad():
    for i, (batch_x, label) in enumerate(tqdm(public_test_loader)):
        batch_x = batch_x.float().to(device)
        label = label.to(device)
        # padding_mask = padding_mask.float().to(device)

        outputs = exp.swa_model(batch_x, None, None, None)

        pred = outputs.detach().cpu()

        preds.append(outputs.detach())
        trues.append(label)

preds = torch.cat(preds, 0)
trues = torch.cat(trues, 0)

probs = torch.nn.functional.softmax(
# probs = torch.nn.functional.sigmoid(
    preds
)  # (total_samples, num_classes) est. prob. for each class and sample

trues_onehot = (torch.nn.functional.one_hot(trues.reshape(-1,).to(torch.long),
                num_classes=CONFIG['num_class'],).float().cpu().numpy())

print(trues_onehot.shape)
predictions = (
    torch.argmax(probs, dim=1).cpu().numpy()
)  # (total_samples,) int class index for each sample
probs = probs.cpu().numpy()
trues = trues.flatten().cpu().numpy()

In [ ]:
# softmax
f1 = f1_score(trues, predictions, average="binary")
auroc = roc_auc_score(trues_onehot, probs)
# auroc = roc_auc_score(trues, probs)
mcc = matthews_corrcoef(trues, predictions)
cpi = (0.25 * f1) + (0.25 * auroc) + (0.5 * mcc)

print(f"F1 score: {f1}")
print(f"AUROC score: {auroc}")
print(f"MCC score: {mcc}")
print(f"CPI score: {cpi}")
print()
print(f"0개수: {len(np.where(predictions[:]==0)[0])}")
print(f"1개수: {len(np.where(predictions[:]==1)[0])}")

In [ ]:
# df = pd.DataFrame({"0":probs[:,0], "1":probs[:,1]})
df = pd.DataFrame({"Probability":probs[:,1]})

# save_xlsx = pd.ExcelWriter(f"./RTFACT_Public_Possibility_{CONFIG['model_id']}.xlsx")
# df.to_excel(save_xlsx, index = False) # xlsx 파일로 변환
# save_xlsx.save() #xlsx 파일로 저장

# ExcelWriter 사용
with pd.ExcelWriter(f"../../Submission results/private/RTFACT_241014_Private_Possibility_{CONFIG['model_id']}_Softmax.xlsx", engine='openpyxl') as writer:
# with pd.ExcelWriter(f"../../Submission results/private/RTFACT_241014_Private_Possibility_{CONFIG['model_id']}_Sigmoid.xlsx", engine='openpyxl') as writer:
    df.to_excel(writer, sheet_name='Sheet1', index=False)